In [1]:
from datasets import load_from_disk, load_dataset

BASE_URL = "/home/pranav-pc/projects/OpenTransformer/multiformer/"
data = load_from_disk(
    BASE_URL + "data/interim/TinyStories_65>tk>1024.hf",
)


# import plotly.express as px

# fig = px.histogram(data['len']);
# fig.show()

In [2]:
from src.tokenize.tokenizer import Tokenizer

TOKENIZER_CHECKPOINT = (
    "/home/pranav-pc/projects/OpenTransformer/multiformer/tokenizer_checkpoints/"
)

tokenizer = Tokenizer(TOKENIZER_CHECKPOINT)

In [4]:
tokenizer.eos_id()

2

In [3]:
# import torch
# x = torch.randint(0,109,(100,)).sort().values.tolist()
# print(x)

# bucket = []
# start = 0
# end = len(x) - 1

# while start < end:
#     max_seq_len = 324
#     if x[end] >= max_seq_len:
#         bucket.append(end)
#     else:
#         if x[end] + x[start] > max_seq_len:
#             bucket.append(end)
#         else:
#             small_bucket = [end,start]
#             max_seq_len -= 2
#             while sum(x[i] for i in small_bucket) + x[start + 1] <= max_seq_len:
#                 start += 1
#                 max_seq_len -= 2
#                 small_bucket.append(start)
#             bucket.append(small_bucket)

#             start += 1
#     print(max_seq_len)
#     end -= 1
# if start==end:
#     bucket.append(start)

In [4]:
def pack_seq(
    example,
    min_seq_len=65,
    max_seq_len=1024,
    delimiter_id=29889,
    eos_id=2,
    para_separator=13,
    sos_id=1,
):
    count = example["len"]
    idx = example["idx"]
    bucket = []
    overflow_bucket = []
    start = 0
    end = len(count) - 1

    while start < end:
        max_seq_len = max_seq_len
        if count[end] >= max_seq_len:
            index = [
                indx
                for indx, i in enumerate(idx[end][:max_seq_len])
                if i == delimiter_id
            ]
            if len(index):
                index = index[-1] + 1
                trim_token = idx[end][:index] + [eos_id]
                bucket.append(trim_token)
                # overflow
                overflow_token = [sos_id] + idx[end][index:]
                if len(overflow_token) > min_seq_len:
                    while len(overflow_token) > max_seq_len:
                        index = [
                            indx
                            for indx, i in enumerate(overflow_token[:max_seq_len])
                            if i == delimiter_id
                        ]
                        if len(index):
                            index = index[-1] + 1
                            trim_token = overflow_token[:index] + [eos_id]
                            overflow_bucket.append(trim_token)
                            overflow_token = [sos_id] + overflow_token[index:]
                        else:
                            break
                    overflow_bucket.append(overflow_token)

            else:
                print("Discarding sample at index : ", end)

        else:
            if count[end] + count[start] > max_seq_len:
                bucket.append(idx[end])
            else:
                small_bucket = []
                idx[end].pop()  # remove eod_id
                idx[start].pop()
                small_bucket.extend(idx[end])
                small_bucket.extend(
                    [para_separator, para_separator]
                )  # adding /n/n after each entry
                small_bucket.extend(idx[start])

                # max_seq_len -= 2
                while (start < end) and (
                    len(small_bucket) + count[start + 1] <= (max_seq_len - 2)
                ):
                    start += 1
                    # max_seq_len -= 2
                    small_bucket.extend([para_separator, para_separator])
                    idx[start].pop()
                    small_bucket.extend(idx[start])
                small_bucket.append(eos_id)
                bucket.append(small_bucket)

                start += 1
        end -= 1
    if start == end:
        bucket.append(idx[end])

    bucket = bucket + [None] * (
        len(idx) - len(bucket)
    )  # hf dataset lib expect map func to return same dim as input i.e len(bucket) == len(idx)
    overflow_bucket = overflow_bucket + [None] * (len(idx) - len(overflow_bucket))
    return {"packed": bucket, "overflow": overflow_bucket}

In [6]:
data = data.map(lambda example: {"len": len(example["idx"])})

Map:   0%|          | 0/2128578 [00:00<?, ? examples/s]

In [5]:
data = data.map(pack_seq, batched=True, batch_size=int(1e5), num_proc=25)

Map (num_proc=25):   0%|          | 0/2119253 [00:00<?, ? examples/s]

In [6]:
packed_data = data.remove_columns(["idx", "len", "overflow"]).rename_column(
    "packed", "tokens"
)
overflow_data = data.remove_columns(["idx", "len", "packed"]).rename_column(
    "overflow", "tokens"
)

In [9]:
packed_data = packed_data.filter(
    lambda x: x["tokens"], num_proc=25, batched=True, batch_size=int(1e5)
)
overflow_data = overflow_data.filter(
    lambda x: x["tokens"], num_proc=25, batch_size=int(1e5)
)

Filter (num_proc=25):   0%|          | 0/2112144 [00:00<?, ? examples/s]

Filter (num_proc=25):   0%|          | 0/1543 [00:00<?, ? examples/s]

In [19]:
tokenizer.decode_ids(overflow_data[1500]["tokens"]).strip()

'"Hey, you two!" the big boy shouted. "That is my spade! Give it back!"\n\nTom and Mia looked up. They saw the big boy. He was bigger and meaner than them. He wanted his spade back.\n\nWhat do you think will happen next?'

In [20]:
print(tokenizer.decode_ids(packed_data[6100]["tokens"]))

Once upon a time there were two friends, Sam and Jack. Sam had a big eraser and Jack wanted it. He kept demanding it from Sam but Sam refused.

"No way! It's mine," said Sam.

"But I want it," whined Jack.

Sam had a silly idea. He said, "I'll give it to you if you give me five hugs."

Jack thought about it and then agreed. So he hugged Sam five times and then Sam gave him the eraser. After that, they both played together smiling and laughing. They had lots of fun!


In [3]:
data_validation = load_dataset(
    BASE_URL + "data/downloads/TinyStories",
    split="validation",
)

In [12]:
print(data_validation[6150]["text"])

Once upon a time, there was a little boy named Timmy. Timmy loved cars and he had a blue toy car that he played with all the time. One day, Timmy's mom took him to the store to buy some things. While they were there, Timmy saw a big red car that he really wanted.

"Mommy, I want that car," said Timmy.

"But Timmy, we don't have enough money to buy it," replied his mom.

Timmy was sad, but then he had an idea. He said, "Mommy, can we speed up and get more money?"

His mom laughed and said, "No Timmy, we can't do that. But we can save up our money and maybe one day we can buy the red car."

Timmy was happy with his mom's answer and he went back to playing with his blue toy car.


In [10]:
data_validation

Dataset({
    features: ['text'],
    num_rows: 21990
})

In [13]:
dataset = data.map(lambda example: {"len": len(example["idx"])})

Map:   0%|          | 0/2128578 [00:00<?, ? examples/s]

In [ ]:
def dummy(example)

In [18]:
dataset.map(
    dummy,
)

TypeError: Dataset.map() got an unexpected keyword argument 'shuffle'

In [9]:
tokenizer.piece_to_id("<0x0A>")

13